# Лабораторна робота 2

Розробка  програмного  забезпечення  для  реалізації  ймовірнісної 
нейронної мережі PNN

In [2]:
import math

X_train = [[1., 1.],[1.,2],[2.,1.],[10.,10.]]
x = [[1.,0],[3.,4], [12., 10]]
radius = 0.3


def find_activity(x, a):
    Q = math.e**(-((x-a)**2)/(radius**2))
    return Q 

def learn_PNN(X_train, x):

    n = len(X_train)
    param_num = len(x)
    y_example = [0]*n

    for i in range(n):
        sum_elem = 0
        for j in range(param_num):
            sum_elem += find_activity(X_train[i][j], x[j])
        y_example[i] = sum_elem

    y_A = sum(y_example[:3])/len(y_example[:3])
    y_B = y_example[-1]/1

    if y_A>= y_B:
        answer = "A"
    else:
        answer = "B"
    return answer

for example in x:
    class_type = learn_PNN(X_train, example)
    print('{} is {} class'.format(example, class_type))

[1.0, 0] is A class
[3.0, 4] is A class
[12.0, 10] is B class


# Лабораторна робота 4

In [1]:
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import SGD
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# Розмір міні-вибірки
batch_size = 32
# Кількість класів зображень
nb_classes = 10
# Кількість епох навчання 
nb_epoch = 25
# Розмір зображення
img_rows, img_cols = 32, 32
# Кількість каналів: RGB
img_channels = 3
# Нормалізація даних
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)
# Створення нейромережевої моделі
model = Sequential()

# Перший шар згортки
model.add(Conv2D(32, (3, 3), padding='same',
                        input_shape=(32, 32, 3), activation='relu'))
# Друний шар згортки
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
# Перший шар субдискретизаії 
model.add(MaxPooling2D(pool_size=(2, 2)))
# Перший шар Dropout
model.add(Dropout(0.25))

# Третій шар згортки
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# Четвертий шар згортки
model.add(Conv2D(64, (3, 3), activation='relu'))
# Другий шар субдисктеризації
model.add(MaxPooling2D(pool_size=(2, 2)))
# Другий шар Dropout
model.add(Dropout(0.25))
# Шар перетворення вхідних даних 
model.add(Flatten())
# Повнозв’язний шар
model.add(Dense(512, activation='relu'))
# Третій шар Dropout
model.add(Dropout(0.5))
# Вихідний шар 
model.add(Dense(nb_classes, activation='softmax'))
# Параметри оптимізації
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/pollymorphism/miniconda3/lib/python3.7/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/home/pollymorphism/miniconda3/lib/python3.7/imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.10.0: cannot open shared object file: No such file or directory


In [ ]:
%%time

# Навчання  моделі
model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              validation_split=0.1,
              shuffle=True,
              verbose=2)
# Оцінка якості навчання на тестових даних
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy on test data: %.2f%%" % (scores[1]*100))
# Збереження моделі
model.save('my_model.h5')
numpy.savez('test_data.npz', X_test=X_test, y_test=y_test)



In [ ]:
import numpy as np
from keras.utils import np_utils
from keras.models import load_model
%matplotlib inline
import matplotlib.pyplot as plt
data = np.load('test_data.npz')
X_test = data['X_test']
y_test = data['y_test']
model = load_model('my_model.h5')    
for i in range(9):
    plt.subplot(350 + 1 + i) #\
    plt.imshow(X_test[i], interpolation='bicubic')
plt.show()
plt.imshow(X_test[1], interpolation='bicubic')
plt.show()

X_test = X_test.astype('float32')
X_test /= 255

Y_test = np_utils.to_categorical(y_test, 10)

labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
counter_labels = dict(enumerate(labels, 0))
print(counter_labels)

pred = model.predict(X_test)
y = np.argmax(pred, axis=-1)
y[:20]
plt.imshow(X_test[8], interpolation='bicubic')
plt.show()
print('Recognized object:', counter_labels[y[index]])